In [1]:
!pip install torch torchvision torchaudio
!pip install torch-geometric numpy scipy mne pandas scikit-learn
!pip install mne
!pip install seiz_eeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import mne

from pathlib import Path
from seiz_eeg.dataset import EEGDataset
from torch_geometric.data import Data, Dataset as PyGDataset, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch.utils.data import WeightedRandomSampler
from scipy import signal
from scipy.signal import welch

### Data Loading  
Reads the pre‑windowed EEG metadata from parquet files and initializes `EEGDataset` instances, applying any time‑or frequency‑domain transforms.


In [3]:
# EEG parameters used later for the creation for the adjacency matric
CH_NAMES   = [
    'Fp1','Fp2','F7','F3','Fz','F4','F8',
    'T3','C3','Cz','C4','T4','T5','P3',
    'Pz','P4','T6','O1','O2'
]

# data Loading (using `EEGDataset`)
# we read the pre-windowed segments from parquet, then wrap them into graphs

DATA_ROOT = Path("/content/drive/MyDrive/EPFL/NML")

# one row = one 12s window
clips_tr = pd.read_parquet(DATA_ROOT / "train" / "train" / "segments.parquet")
clips_te = pd.read_parquet(DATA_ROOT / "test"  / "test" / "segments.parquet")

bp_filter = signal.butter(4, (0.5, 30), btype="bandpass", output="sos", fs=250)

# filtering for the signals (given at example.ipynb)
def time_filtering(x: np.ndarray) -> np.ndarray:
    """Filter signal in the time domain"""
    return signal.sosfiltfilt(bp_filter, x, axis=0).copy()


def fft_filtering(x: np.ndarray) -> np.ndarray:
    """Compute FFT and only keep"""
    x = np.abs(np.fft.fft(x, axis=0))
    x = np.log(np.where(x > 1e-8, x, 1e-8))

    win_len = x.shape[0]
    # Only frequencies b/w 0.5 and 30Hz
    return x[int(0.5 * win_len // 250) : 30 * win_len // 250]

# create the EEGDataset instances
dataset_tr = EEGDataset(
    clips_tr,
    signals_root   = DATA_ROOT / "train" / "train",
    signal_transform=fft_filtering,
    prefetch=True
)

dataset_te = EEGDataset(
    clips_te,
    signals_root   = DATA_ROOT / "test" / "test",
    signal_transform=fft_filtering,
    prefetch=True,
    return_id=True
)

print(f"Loaded {len(dataset_tr):,} training windows, {len(dataset_te):,} test windows.")

Loaded 12,993 training windows, 3,614 test windows.


### Preprocessing & Graph Construction  
Loads from the distances from the given distances_3d.csv file, pivots it into a 19×19 distance matrix, applies an RBF kernel to convert distances into similarities, and thresholds to build the adjacency matrix.


In [4]:
def load_adjacency(dist_csv, ch_names, threshold_pct=75):
    """
    Read the 3-columns [from,to,distance] of distances_3d.csv and build a symmetric adjacency:
    """
    # read and pivot
    df = pd.read_csv(dist_csv)
    dmat = df.pivot(index='from', columns='to', values='distance')
    dmat = dmat.reindex(index=ch_names, columns=ch_names)
    dist = dmat.values.astype(float)

    # zero the diagonal
    np.fill_diagonal(dist, 0.0)

    # mirror known entries to get symmetric matrix
    mask = np.isnan(dist)
    dist[mask] = dist.T[mask]

    # fill any remaining NaNs with the max so that missing pairs become “very far apart”
    max_dist = np.nanmax(dist)
    dist[np.isnan(dist)] = max_dist

    # build RBF weights
    sigma = dist.mean()
    W = np.exp(-dist**2 / (2 * sigma**2))

    # sparsify by zeroing out the weakest edges
    cutoff = np.percentile(W, threshold_pct)
    W[W < cutoff] = 0.0

    # zero the diagonal again
    np.fill_diagonal(W, 0.0)

    return W

distances_csv = DATA_ROOT / "distances_3d.csv"
A = load_adjacency(distances_csv, CH_NAMES, threshold_pct=30) # changed the threshold to a lower value otherwise I was getting many 0s (only very strong connections were considered)
print("Adjacency shape:", A.shape, "  density:", (A>0).mean())
print(A)

Adjacency shape: (19, 19)   density: 0.9473684210526315
[[0.         0.43858603 0.43858603 0.43858603 0.43858603 0.43858603
  0.43858603 0.43858603 0.43858603 0.43858603 0.43858603 0.43858603
  0.43858603 0.43858603 0.43858603 0.43858603 0.43858603 0.43858603
  0.43858603]
 [0.43858603 0.         0.43858603 0.43858603 0.43858603 0.43858603
  0.43858603 0.43858603 0.43858603 0.43858603 0.43858603 0.43858603
  0.43858603 0.43858603 0.43858603 0.43858603 0.43858603 0.43858603
  0.43858603]
 [0.43858603 0.43858603 0.         0.93576703 0.43858603 0.64896094
  0.58308472 0.92430892 0.83832074 0.43858603 0.52317844 0.47450356
  0.75218863 0.67583695 0.43858603 0.46869762 0.4385897  0.58304998
  0.47448915]
 [0.43858603 0.43858603 0.93576703 0.         0.43858603 0.78010715
  0.64896094 0.83036187 0.89836861 0.43858603 0.65241855 0.52819134
  0.67583695 0.68944871 0.43858603 0.53784387 0.46869762 0.5458095
  0.47460245]
 [0.43858603 0.43858603 0.43858603 0.43858603 0.         0.43858603
  0.4

### PyG Dataset Wrapper  
Defines `GraphFromEEG`, which takes each transformed EEG window, computes per‑channel features (mean, variance, peak-to-peak, zero-crossing rate etc.), and uses the fixed graph topology (edges + weights) to produce `torch_geometric.data.Data` objects.


In [7]:
# Hyperparameters
batch_size    = 32
epochs        = 20
learning_rate = 1e-3

SFREQ = 250  # Hz, matches the dataset’s sampling rate

class GraphFromEEG(PyGDataset):
    def __init__(self, eeg_ds, adj, is_test=False):
        super().__init__()
        self.eeg_ds   = eeg_ds
        self.is_test  = is_test

        rows, cols        = np.nonzero(adj > 0)
        self.edge_index   = torch.tensor([rows, cols], dtype=torch.long)
        self.edge_weight  = torch.tensor(adj[rows, cols], dtype=torch.float)

    def len(self):
        return len(self.eeg_ds)

    def get(self, idx):
        arr, meta = self.eeg_ds[idx]
        if self.is_test:
            signal, sid = arr, meta
            label = None
        else:
            signal, label = arr, meta
            sid = None

        # signal: (n_time_bins, n_channels)
        # Compute 9 features per channel

        # 1) mean
        mean_ = signal.mean(axis=0)

        # 2) variance
        var_  = signal.var(axis=0)

        # 3) peak-to-peak
        ptp_  = np.ptp(signal, axis=0)

        # 4) zero-crossing rate
        zcr_  = np.mean(np.diff(np.sign(signal), axis=0) != 0, axis=0)

        # 5) PSD via Welch
        freqs, psd = welch(signal, fs=SFREQ, axis=0)

        def bandpower(pxx, freqs, fmin, fmax):
            mask = (freqs >= fmin) & (freqs <= fmax)
            return pxx[mask].mean(axis=0)

        # 6–10) Bandpower in δ (1–4), θ (4–8), α (8–12), β (12–30), γ (30–45)
        delta = bandpower(psd, freqs, 1, 4)
        theta = bandpower(psd, freqs, 4, 8)
        alpha = bandpower(psd, freqs, 8, 12)
        beta  = bandpower(psd, freqs, 12, 30)
        gamma = bandpower(psd, freqs, 30, 45)

        # stack into (n_channels, 9) feature matrix
        features = np.stack([mean_, var_, ptp_, zcr_,
                             delta, theta, alpha, beta, gamma], axis=1)

        x = torch.tensor(features, dtype=torch.float)

        y = torch.tensor([label], dtype=torch.long) if label is not None else None

        data = Data(x=x,
                    edge_index=self.edge_index,
                    edge_attr=self.edge_weight,
                    y=y)

        # keep index for later id lookup
        data.idx = torch.tensor([idx], dtype=torch.long)
        return data

graph_tr = GraphFromEEG(dataset_tr, A, is_test=False)
graph_te = GraphFromEEG(dataset_te, A, is_test=True)

loader_tr = DataLoader(graph_tr, batch_size=batch_size, shuffle=True)
loader_te = DataLoader(graph_te, batch_size=batch_size, shuffle=False)

print(f"Graphified train size: {len(graph_tr)}, test size: {len(graph_te)}")


Graphified train size: 12993, test size: 3614


### DataLoader with balancing
Instantiate `DataLoader` over our training graph dataset—using a `WeightedRandomSampler` to balance seizure vs. non‑seizure windows during training


In [8]:
# Balance the labels by giving more weight to the minority class
# Difference in the number of samples per class: [10476, 2517]

train_labels = [data.y.item() for data in graph_tr]
counts = np.bincount(train_labels)
weights = 1.0 / counts    # gives more weight to the minority class

# sample‐wise weight vector
sample_weights = np.array([weights[l] for l in train_labels])
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# rebuild the train loader
loader_tr = DataLoader(
    graph_tr,
    batch_size=batch_size,
    sampler=sampler,
    num_workers=2
)


### Model Definition & Training
Builds the GCN model (`EEG_GCN`) with two graph‐convolution layers followed by global mean pooling and a linear layer for binary classification also trains the GCN over 20 epochs using cross‑entropy and prints train loss and accuracy.

In [9]:
# Model & Training

class EEG_GCN(torch.nn.Module):
    def __init__(self, in_feats=1, hid_feats=64, num_classes=2):
        super().__init__()
        self.conv1 = GCNConv(in_feats, hid_feats)
        self.conv2 = GCNConv(hid_feats, hid_feats)
        self.lin   = torch.nn.Linear(hid_feats, num_classes)

    def forward(self, data):
        x, ei, ew, batch = data.x, data.edge_index, data.edge_attr, data.batch
        x = F.relu(self.conv1(x, ei, edge_weight=ew))
        x = F.relu(self.conv2(x, ei, edge_weight=ew))
        x = global_mean_pool(x, batch)
        return self.lin(x)

def train_epoch(model, loader, opt, device):
    model.train()
    total = 0
    for data in loader:
        data = data.to(device)
        opt.zero_grad()
        loss = F.cross_entropy(model(data), data.y.view(-1))
        loss.backward()
        opt.step()
        total += loss.item() * data.num_graphs
    return total / len(loader.dataset)

def evaluate(model, loader, device):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            preds = model(data).argmax(dim=1)
            correct += (preds == data.y.view(-1)).sum().item()
            total   += data.num_graphs
    return correct / total

# Train & Validation

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model     = EEG_GCN(in_feats = 9).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(1, epochs+1):
    loss   = train_epoch(model, loader_tr, optimizer, device)
    acc    = evaluate(model, loader_tr, device)
    print(f"Epoch {epoch:02d}: Train Loss = {loss:.4f}, Train Acc = {acc:.4f}")

Epoch 01: Train Loss = 0.6416, Train Acc = 0.6496
Epoch 02: Train Loss = 0.6250, Train Acc = 0.6128
Epoch 03: Train Loss = 0.6131, Train Acc = 0.6596
Epoch 04: Train Loss = 0.6073, Train Acc = 0.6550
Epoch 05: Train Loss = 0.5965, Train Acc = 0.6870
Epoch 06: Train Loss = 0.5932, Train Acc = 0.6699
Epoch 07: Train Loss = 0.5811, Train Acc = 0.7021
Epoch 08: Train Loss = 0.5807, Train Acc = 0.6961
Epoch 09: Train Loss = 0.5751, Train Acc = 0.6744
Epoch 10: Train Loss = 0.5744, Train Acc = 0.7161
Epoch 11: Train Loss = 0.5663, Train Acc = 0.7130
Epoch 12: Train Loss = 0.5666, Train Acc = 0.7156
Epoch 13: Train Loss = 0.5513, Train Acc = 0.6859
Epoch 14: Train Loss = 0.5549, Train Acc = 0.6987
Epoch 15: Train Loss = 0.5583, Train Acc = 0.7016
Epoch 16: Train Loss = 0.5558, Train Acc = 0.7125
Epoch 17: Train Loss = 0.5525, Train Acc = 0.7145
Epoch 18: Train Loss = 0.5530, Train Acc = 0.7288
Epoch 19: Train Loss = 0.5539, Train Acc = 0.7213
Epoch 20: Train Loss = 0.5484, Train Acc = 0.7300


### Test Prediction & Submission  
Run inference on the test loader, map each prediction back to the original window IDs and write out a Kaggle‐compatible CSV of `id,label` rows.


In [10]:
## Test & Submission

model.eval()
all_idxs, all_preds = [], []

with torch.no_grad():
    for batch in loader_te:
        # batch.idx is a tensor of shape [batch_size] giving the original idx
        all_idxs.extend(batch.idx.cpu().tolist())
        batch = batch.to(device)
        logits = model(batch)
        preds = logits.argmax(dim=1).cpu().tolist()
        all_preds.extend(preds)

all_ids = clips_te.index.tolist()

# check
assert len(all_ids) == len(all_preds)

# write submission
submission = pd.DataFrame({'id': all_ids, 'label': all_preds})
submission.to_csv('submission.csv', index=False)
print(f"Saved submission.csv with {len(submission)} rows")

Saved submission.csv with 3614 rows
